In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import f1_score

In [3]:
data = pd.read_csv("../input/weather-dataset-rattle-package/weatherAUS.csv")
print(data.shape)
data.head(4)

In [4]:
data.isna().sum()

In [5]:
data.drop(['Date'],axis=1,inplace=True)

In [6]:
data.isna().sum()

In [7]:
data.dtypes

In [8]:
catg = [c for c in data.dtypes.index if data.dtypes[c] == 'object']
catg

In [9]:
num = [c for c in data.dtypes.index if c not in catg]
num

In [10]:
def imputer(data, colsNum, colsCatg):
    for c in colsNum:
        data[c]= data[c].fillna(data[c].mean())
    for c in colsCatg:
        data[c]= data[c].fillna(data[c].mode()[0])

In [11]:
imputer(data,num,catg)

In [12]:
data.isna().sum()

In [13]:
for c in catg:
    print("\n__________",c,"_____________________\n")
    print(data[c].unique())

In [14]:
for c in ['RainToday','RainTomorrow']:
    le=LabelEncoder()
    data[c]=le.fit_transform(data[c])
    print(le.classes_)

In [15]:
data[catg]

In [16]:
pd.get_dummies(data['WindGustDir'])

In [17]:
pd.get_dummies(data['WindDir9am'])

In [18]:
def prefix(c, data, prefix):
    data[c]=data[c].apply(lambda x: prefix + str(x))

In [19]:
prefix('WindGustDir',data,"G")
prefix('WindDir9am',data,"9am")
prefix('WindDir3pm',data,"3am")

In [20]:
data['WindGustDir']

In [21]:
data.head(3)

In [22]:
def oneHot(cols,data):
    for c in cols:
        df=pd.get_dummies(data[c])
        data= pd.concat([data,df],axis =1)
        data.drop(c,axis=1,inplace=True)
    return data
     

In [23]:
ca=['WindDir3pm' , 'WindDir9am', 'WindGustDir', 'Location']
data=oneHot(ca,data)

In [24]:
data

In [25]:
y=data['RainTomorrow']
x=data.drop(['RainTomorrow'],axis =1)

In [26]:
sc= RobustScaler()
x=pd.DataFrame(sc.fit_transform(x),columns=x.columns)

In [27]:
x

# training

In [28]:
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.85)


In [29]:
inputs =tf.keras.Input(shape=(114,))
x=tf.keras.layers.Dense(16,activation='relu')(inputs)
x=tf.keras.layers.Dense(16,activation='relu')(x)
outputs=tf.keras.layers.Dense(2,activation='softmax')(x)

model=tf.keras.Model(inputs = inputs,outputs=outputs)

In [30]:
model.summary()

In [31]:
model.compile(
optimizer ='adam',
loss ='sparse_categorical_crossentropy',
metrics = ['accuracy']
)

In [32]:
epochs =10
batch_size=32

In [33]:
history = model.fit(
X_train,
    y_train,
    validation_split=0.2,
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)

In [34]:
plt.figure(figsize=(14,10))

plt.plot(range(epochs),history.history['loss'],color ='b')
plt.plot(range(epochs),history.history['val_loss'],color='r')

plt.xlabel('epochs')
plt.ylabel('loss')

plt.show()

In [35]:
np.argmin(history.history['val_loss'])

In [36]:
print(f"Model Accuracy: {model.evaluate(X_test, y_test, verbose=0)[1]}")


In [37]:
y.sum() / len(y)

In [38]:
y_pred = model.predict(X_test)

In [39]:
y_pred

In [40]:
y_test

In [41]:
y_pred = list(map(lambda x: np.argmax(x), y_pred))

In [42]:
print(f"Model F1 Score: {f1_score(y_test, y_pred)}")
